In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device == torch.device('cpu'):
    print('Using cpu')
else:
    n_gpu = torch.cuda.device_count()
    print('Using {} GPUs'.format(torch.cuda.get_device_name(0)))

Using Tesla T4 GPUs


In [2]:
! pip install datasets transformers seqeval

     |████████████████████████████████| 225kB 7.6MB/s 
     |████████████████████████████████| 2.3MB 30.3MB/s 
     |████████████████████████████████| 51kB 9.3MB/s 
     |████████████████████████████████| 112kB 51.3MB/s 
     |████████████████████████████████| 245kB 53.4MB/s 
     |████████████████████████████████| 3.3MB 53.9MB/s 
     |████████████████████████████████| 901kB 49.8MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=46381d6ddedb652206dad910c565a974a88e6a0a34eaf7100af33d1a4bd03111
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval
ERROR: transformers 4.6.1 has requirement huggingface-hub==0.0.8, but you'll have huggingface-hub 0.0.9 which is incompatible.


In [3]:
task = "ner"
batch_size = 8

# Загрузка данных - Request For Quote


In [4]:
!gdown --id 1L6dd0FnYqgn-eoQ-gFnBiNji7R1ul9n_

Downloading...
From: https://drive.google.com/uc?id=1L6dd0FnYqgn-eoQ-gFnBiNji7R1ul9n_
To: /content/NER_RFQ_agg.csv
4.39MB [00:00, 20.5MB/s]


In [5]:
!gdown --id 1-5mE9XjocmyCKGlkpW1YGuCnNsGwQioD

Downloading...
From: https://drive.google.com/uc?id=1-5mE9XjocmyCKGlkpW1YGuCnNsGwQioD
To: /content/NER_RFQ_agg_train.csv
3.48MB [00:00, 13.2MB/s]


In [6]:
!gdown --id 1d4er4I7x4VIwy7BsWFpsPuC2z6aZ3P6s

Downloading...
From: https://drive.google.com/uc?id=1d4er4I7x4VIwy7BsWFpsPuC2z6aZ3P6s
To: /content/NER_RFQ_agg_test.csv
100% 909k/909k [00:00<00:00, 5.57MB/s]


In [7]:
import pandas as pd
data = pd.read_csv('NER_RFQ_agg.csv', converters={'tokens': eval, 'ner_tags': eval})

In [8]:
all_tags = set()
for _, row in data.iterrows():
    for tag in row.ner_tags:
        all_tags.add(tag)
all_tags = sorted(list(all_tags))

In [9]:
tag_to_idx = {t: i for i, t in enumerate(all_tags)}

In [10]:
label_list = {i: t for i, t in enumerate(all_tags)}

In [11]:
train = pd.read_csv('NER_RFQ_agg_train.csv', converters={'tokens': eval, 'ner_tags': eval})
test = pd.read_csv('NER_RFQ_agg_test.csv', converters={'tokens': eval, 'ner_tags': eval})

In [12]:
train.ner_tags = train.ner_tags.apply(lambda x : [tag_to_idx[label] for label in x])
test.ner_tags = test.ner_tags.apply(lambda x : [tag_to_idx[label] for label in x])

In [13]:
train.to_pickle('train.pkl')
test.to_pickle('test.pkl')

In [14]:
from datasets import load_dataset, load_metric

In [15]:
datasets = load_dataset('pandas', data_files={'train' : 'train.pkl', 'test' : 'test.pkl'})

Using custom data configuration default-40312a13f1f9edb7


Dataset pandas downloaded and prepared to /root/.cache/huggingface/datasets/pandas/default-40312a13f1f9edb7/0.0.0/3547ff13f14ebf30dc8f11ab416dcf7721c18de2eb7e054bf9c80b88e765a791. Subsequent calls will reuse this data.


In [16]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 953
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 239
    })
})

# Токенизатор


In [17]:
from transformers import AutoTokenizer
model_checkpoint = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [18]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [19]:
label_all_tokens = False

In [20]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [21]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Warning : есть последовательности больше, чем максимально возможная в BERT - 512

In [22]:
pd.Series(tokenized_datasets["train"]['input_ids']).apply(lambda x : len(x)).describe()

count     953.000000
mean      727.997901
std       655.440163
min       144.000000
25%       384.000000
50%       557.000000
75%       850.000000
max      7082.000000
dtype: float64

In [23]:
pd.Series(tokenized_datasets["test"]['input_ids']).apply(lambda x : len(x)).describe()

count     239.000000
mean      748.744770
std       618.354517
min       202.000000
25%       385.000000
50%       568.000000
75%       898.500000
max      6996.000000
dtype: float64

In [24]:
train_tokenized = tokenized_datasets["train"].to_pandas()
test_tokenized = tokenized_datasets["test"].to_pandas()

In [25]:
train_tokenized.head(1)

,attention_mask,id,input_ids,labels,ner_tags,token_type_ids,tokens
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",123783R.msg,"[101, 11836, 95269, 51136, 267, 132, 256, 8461...","[-100, 2, 2, -100, -100, -100, -100, -100, 2, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[name, 123783r.msg, yuliya, a., kondratova, <y..."


In [26]:
test_tokenized.head(1)

,attention_mask,id,input_ids,labels,ner_tags,token_type_ids,tokens
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",123670R.msg,"[101, 11836, 97415, 21503, 267, 132, 256, 8461...","[-100, 2, 2, -100, -100, -100, -100, -100, 2, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[name, 123670r.msg, =?koi8-r?b?68ht2nhoz9fbiod..."


In [27]:
CHUNCK_SIZE = 512

def lists_to_chuncks(row):
    n = CHUNCK_SIZE - 2
    new_row = row.copy()
    for key in ['attention_mask', 'input_ids', 'labels', 'token_type_ids']:
        FIRST = new_row[key][0]
        LAST = new_row[key][-1]
        new_row[key] = new_row[key][1:-1]
        new_row[key] = [[FIRST] + list(new_row[key][i:i + n]) + [LAST] for i in range(0, len(new_row[key]), n)]
    return new_row

In [28]:
train_tokenized = train_tokenized.drop(columns = ['ner_tags', 'tokens']).apply(lists_to_chuncks, axis=1)
train_tokenized = train_tokenized.set_index(['id']).apply(pd.Series.explode).reset_index()

In [29]:
test_tokenized = test_tokenized.drop(columns = ['ner_tags', 'tokens']).apply(lists_to_chuncks, axis=1)
test_tokenized = test_tokenized.set_index(['id']).apply(pd.Series.explode).reset_index()

In [30]:
train_tokenized.to_pickle('train_tokenized.pkl')
test_tokenized.to_pickle('test_tokenized.pkl')

In [31]:
tokenized_datasets = load_dataset('pandas', data_files={'train' : 'train_tokenized.pkl', 'test' : 'test_tokenized.pkl'})

Using custom data configuration default-1d5d3695aaf4169a


Dataset pandas downloaded and prepared to /root/.cache/huggingface/datasets/pandas/default-1d5d3695aaf4169a/0.0.0/3547ff13f14ebf30dc8f11ab416dcf7721c18de2eb7e054bf9c80b88e765a791. Subsequent calls will reuse this data.


In [32]:
pd.Series(tokenized_datasets["train"]['input_ids']).apply(lambda x : len(x)).describe()

count    1801.000000
mean      386.162132
std       158.276138
min         5.000000
25%       271.000000
50%       482.000000
75%       512.000000
max       512.000000
dtype: float64

In [33]:
pd.Series(tokenized_datasets["test"]['input_ids']).apply(lambda x : len(x)).describe()

count    452.000000
mean     396.849558
std      143.963162
min        5.000000
25%      302.000000
50%      487.000000
75%      512.000000
max      512.000000
dtype: float64

Теперь все последовательности нужной длины

# Модель


# Обучение

In [34]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Warning : модель надо обучить на наших данных

In [35]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [36]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [37]:
metric = load_metric("seqeval")

In [38]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [39]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [40]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.075713,0.543718,0.664870,0.598222,0.973863
2,No log,0.057096,0.708904,0.743935,0.725997,0.983990
3,0.074600,0.054344,0.744646,0.812219,0.776966,0.985490


TrainOutput(global_step=678, training_loss=0.061411801341009, metrics={'train_runtime': 668.9829, 'train_samples_per_second': 1.013, 'total_flos': 770380182417600.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 2052751360, 'init_mem_gpu_alloc_delta': 709342720, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 136089600, 'train_mem_gpu_alloc_delta': 2196961280, 'train_mem_cpu_peaked_delta': 278351872, 'train_mem_gpu_peaked_delta': 6504170496})

# Сохранение модели


In [41]:
import pickle
with open("RFQ_BERT[DeepPavlov_rubert_base_cased].pkl", 'wb') as f:
    pickle.dump(model, f)

# Валидация

In [42]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_accuracy': 0.9854900280461203,
 'eval_f1': 0.7769660507090674,
 'eval_loss': 0.05434383079409599,
 'eval_mem_cpu_alloc_delta': 1257472,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 294635520,
 'eval_precision': 0.7446457990115322,
 'eval_recall': 0.8122192273135669,
 'eval_runtime': 17.9478,
 'eval_samples_per_second': 25.184}

In [43]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'GoodsString': {'f1': 0.7769660507090674,
  'number': 1113,
  'precision': 0.7446457990115322,
  'recall': 0.8122192273135669},
 'overall_accuracy': 0.9854900280461203,
 'overall_f1': 0.7769660507090674,
 'overall_precision': 0.7446457990115322,
 'overall_recall': 0.8122192273135669}

In [44]:
pd.DataFrame.from_dict(results).transpose().iloc[:-4, :-1]

,f1,number,precision
GoodsString,0.776966,1113.0,0.744646


In [45]:
pd.DataFrame.from_dict(results).transpose().iloc[-4:, 0].rename('result')

overall_precision    0.744646
overall_recall       0.812219
overall_f1           0.776966
overall_accuracy     0.985490
Name: result, dtype: float64

In [46]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 10.1MB/s 


In [48]:
print(metrics.flat_classification_report(true_labels, true_predictions, labels=all_tags[:-1], digits=3))

               precision    recall  f1-score   support

B-GoodsString      0.834     0.867     0.851      1109
I-GoodsString      0.846     0.864     0.855      1718

    micro avg      0.841     0.866     0.853      2827
    macro avg      0.840     0.866     0.853      2827
 weighted avg      0.842     0.866     0.853      2827

